In [4]:
# =============================================================================
# 步驟 0：導入必要的函式庫
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, r2_score
import lightgbm as lgb
import warnings

warnings.filterwarnings('ignore', category=FutureWarning)
print("函式庫導入完成。\n")


函式庫導入完成。



In [6]:
# =============================================================================
# 步驟 1：資料載入與視覺化設定
# =============================================================================
# --- 設定 Matplotlib 顯示中文字體 ---
try:
    plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] # Windows
except:
    try:
        plt.rcParams['font.sans-serif'] = ['Arial Unicode MS'] # Mac
    except:
        print("警告：未找到指定的中文字體，圖表標題可能顯示為亂碼。")
plt.rcParams['axes.unicode_minus'] = False

# --- 載入資料 ---
file_path = u'2023資料剖析後內容1~8月芝山到天母0609整理二版.csv' 
try:
    df = pd.read_csv(file_path, encoding='utf-8-sig')
    print(f"成功使用 'utf-8-sig' 編碼讀取檔案: {file_path}")
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='big5')
    print(f"成功使用 'big5' 編碼讀取檔案: {file_path}")
except FileNotFoundError:
    print(f"錯誤：找不到檔案 '{file_path}'。請確認檔案名稱和路徑是否正確。")
    exit()

print(f"\n原始資料共有 {len(df)} 筆。")



成功使用 'utf-8-sig' 編碼讀取檔案: 2023資料剖析後內容1~8月芝山到天母0609整理二版.csv

原始資料共有 5016 筆。


In [7]:
# =============================================================================
# 步驟 2 & 3：資料清理、預處理與特徵工程 (合併與重構)
# =============================================================================
print("\n--- 開始進行資料處理與特徵工程 ---")

# 使用 .copy() 創建一個明確的副本，避免 SettingWithCopyWarning
df_processed = df.copy()

# 移除檔案結尾可能存在的文字說明行 (判斷依據：Date 欄位為空)
df_processed.dropna(subset=['Date'], inplace=True)

# --- 欄位轉換與新增 ---
# 2.1 轉換騎乘時間為秒數
def parse_time_to_seconds(time_str):
    try:
        return pd.to_timedelta(time_str).total_seconds()
    except (ValueError, TypeError):
        return np.nan # 返回 NumPy 的 NaN，便於後續處理

df_processed['TotalSeconds'] = df_processed['Time2'].apply(parse_time_to_seconds)

# 2.2 轉換日期時間格式
# 先確保 Date 欄位是乾淨的字串，再進行合併
df_processed['Date_str'] = df_processed['Date'].astype(int).astype(str)
df_processed['Datetime'] = pd.to_datetime(df_processed['Date_str'] + ' ' + df_processed['Time1'], format='%Y%m%d %H:%M:%S', errors='coerce')

# --- 進行一次性的資料篩選 ---
# 條件1: TotalSeconds 必須是有效的數值 (非 NaN)
# 條件2: Datetime 必須是有效的日期 (非 NaT)
# 條件3: 騎乘時間必須在合理範圍內 (3 到 60 分鐘)
original_rows = len(df_processed)

df_clean = df_processed[
    (df_processed['TotalSeconds'].notna()) &
    (df_processed['Datetime'].notna()) &
    (df_processed['TotalSeconds'] >= 180) & 
    (df_processed['TotalSeconds'] <= 3600)
].copy() # 再次使用 .copy() 確保是一個新的 DataFrame

print(f"移除了 {original_rows - len(df_clean)} 筆無效或時間極端的資料。")
print(f"資料清理完成，剩餘有效資料 {len(df_clean)} 筆。")


# --- 特徵工程 (在乾淨的資料上進行) ---
df_clean['TotalSeconds'] = df_clean['TotalSeconds'].astype(int)

# 3.1 從 'Datetime' 提取時間特徵
df_clean['Hour'] = df_clean['Datetime'].dt.hour
df_clean['DayOfWeek'] = df_clean['Datetime'].dt.dayofweek
df_clean['Month'] = df_clean['Datetime'].dt.month
df_clean['IsWeekend'] = df_clean['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)

# 3.2 建立尖峰時段特徵
df_clean['IsRushHour'] = df_clean['Hour'].apply(lambda h: 1 if (7 <= h <= 9) or (17 <= h <= 19) else 0)

# 3.3 標準化起點站並進行 One-Hot Encoding
def normalize_exit(stp_name):
    return 'Exit_1' if '1號' in str(stp_name) else 'Exit_2'
df_clean['StartExit'] = df_clean['Stp1'].apply(normalize_exit)

ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype=np.int64)
encoded_features = ohe.fit_transform(df_clean[['StartExit']])
new_feature_names = ohe.get_feature_names_out(['StartExit'])
encoded_df = pd.DataFrame(encoded_features, columns=new_feature_names, index=df_clean.index)
df_final = pd.concat([df_clean, encoded_df], axis=1)

# --- 準備最終用於建模的 DataFrame ---
# 選擇我們需要的特徵欄位和目標欄位
final_columns = ['TotalSeconds', 'Hour', 'DayOfWeek', 'Month', 'IsWeekend', 'IsRushHour'] + list(new_feature_names)
df_model = df_final[final_columns].reset_index(drop=True) # 重設索引

print("\n特徵工程完成。")
print("最終用於建模的 DataFrame 預覽：")
print(df_model.head())




--- 開始進行資料處理與特徵工程 ---
移除了 5016 筆無效或時間極端的資料。
資料清理完成，剩餘有效資料 0 筆。


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.